# Getting Data from the Internet

There is a ton of good data on the internet, but it can be hard to access.  In this lesson we will learn just enough about web scraping to get in trouble.  

**Important**: stay out of trouble!

### Best Practices

1. Don't break anything.  Many rapid requests to smaller sites can overload the host server.
2. Use a published API if possible - it is more robust and usually much easier!
3. Respect the policy published at `robots.txt` 
4. Don't spoof your UserAgent (or try to trick the server into thinking you are a person)
5. Read the Terms of Service for the site and follow it.

# Requests

`requests` is a python package that allows you to use Python to interact with the internet!  There are other packages, but I find `requests` to be much easier to use.

In fact, to get the UCSD home page is a simple as
```
import requests
text = requests.get("https://ucsd.edu").text
```
But before we do that, we need to learn just a little bit more.

# Status Codes

When we request data from a website, the server responds with a HTTP status code.  The most common response is `200` which means things went well.  Other times you will get a different status code saying something else happened - you might be familiar with a `404` which means the page wasn't found.

This great site lists http status codes: [https://httpstat.us/](https://httpstat.us/).

But better yet, it has example sites that return a certain code, so you can test!  So, for example, https://httpstat.us/404 returns a `404`

In [ ]:
import requests

r = requests.get("https://httpstat.us/404")
print(r.status_code)

You can check if the call went ok with `r.ok` which returns a boolean.

After you run the code below, read up on each of the status codes at [https://httpstat.us/](https://httpstat.us/).

In [ ]:
statusCodes = [200, 404, 403, 429]

for statusCode in statusCodes:
    r = requests.get("https://httpstat.us/" + str(statusCode))
    print(str(statusCode) + " ok: " + str(r.ok))

In [ ]:
# Or raise an exception when there is a not-ok status code

r = requests.get("https://httpstat.us/404")
r.raise_for_status()

# Robots.txt

Many sites have a published policy allowing or disallowing automatic access to their site.  It uses a text file `robots.txt` and you can learn more about it [here](https://moz.com/learn/seo/robotstxt).

The code below checks if the `robot.txt` file prohibits you from scraping the site.  Remember the best practices above - just because you aren't prohibited by the robots policy doesn't mean you can scrape the site!

In [ ]:
from urllib.parse import urlparse
import urllib.robotparser

# This code checks the robots.txt file
def canFetch(url):

    parsed_uri = urlparse(url)
    domain = '{uri.scheme}://{uri.netloc}/'.format(uri=parsed_uri)

    rp = urllib.robotparser.RobotFileParser()
    rp.set_url(domain + "/robots.txt")
    try:
        rp.read()
        canFetchBool = rp.can_fetch("*", url)
    except:
        canFetchBool = None
    
    return canFetchBool

In [ ]:
url = "https://slate.com/bullpen/"
canFetch(url)

In [ ]:
url = "http://dsc.ucsd.edu/node/10"
canFetch(url)

# Getting the HTML

Now we can request a website!  Let's see what is on the UCSD Data Science Events page.

In [ ]:
url = "http://dsc.ucsd.edu/node/10"

r = requests.get(url)
    
urlText = r.text

Nchars = 1000
print(urlText[:Nchars]) # Print the first 500 characters
print("\n\n... " + str(len(urlText)-Nchars) + " additional characters")


# Cleaning

Wow, that is gross looking!  It is raw HTML, which the browser uses to make the viewable site.  To process it we can use [Beautiful Soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/).

**Warning** BeautifulSoup has changed quite a bit between versions, so make sure you are looking at documentation for the version you are using (4 here).

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(urlText, 'html.parser')

In [ ]:
# Grab all the links
for link in soup.find_all('a'):
    if link.has_attr('href'):
        print(link['href'])

In [ ]:
# Show the text
print(soup.get_text())

In [ ]:
# That text had too much white space.  Let's try
for string in soup.stripped_strings:
    print(repr(string))

# Next steps

From here you can do a number of different things!

* Scrape individual elements from a site ([example](https://codeburst.io/web-scraping-101-with-python-beautiful-soup-bb617be1f486))
* Pull text down and use NLP from last week (like sentiment analysis)
* Monitor a site daily for changes.
* Use the text to create your own search engine!